### 都知道在torch中用GPU训练速度更快，但是需要把什么放在GPU上，怎么放，怎么判断放上去没有，这是这个文件想要说明的问题

#### 首先，需要把什么放在GPU上？
回答：需要把数据，网络模型，以及部分情况下的损失模型放到GUP上。数据、网络模型后面细说，部分情况下的损失函数指的是什么呢？具体是这样：GPU运算起来更快指的是一些矩阵的运算，而运算的内容就是各种矩阵，其中，数据中有输入矩阵，网络模型中有权重和偏置矩阵，所以都需要放到GPU上，而损失模型大部分时候都含有矩阵，除非某些自定的特殊的损失函数，所以，综上，数据、网络模型都需要放到GPU上，而损失函数需要分情况，当损失函数含有参数或需要跟其他数据进行矩阵运算时，需要将损失函数也放到GPU上，不然不需要。
#### 其次，怎么放？
放之前，需要先使用torch.cuda.is_available()方法来判断设备上是否有GPU设备，如果有，一般来说有两种不同的方法：
一、在创建参数，网络模型和损失模型的时候指定设备，在这之前，我们要先创建cuda设备。
二、先在cpu上创建参数、网络模型和损失函数模型，再将这些内容转移到cuda上，可以使用两个不同的函数分别实现，.to(device)和.cuda()。
#### 第三，怎么检查上面说的这些在不在GPU上？
回答：对于数据来说，比较简单，所有的数据都有两种属性能表明数据在哪，分别是.device和.is_cuda。对于网络模型和损失函数模型，需要通过他们的参数所在的位置来判断在哪，通过next(model.parameters())来获取参数，然后获取参数的.device或.is_cuda属性来判断。
#### 最后，如果电脑上有多个GPU，希望让不同的GPU设备上进行指定数据的运算怎么办？
回答：使用.to(device= )函数，可以将数据、网络模型、损失函数放到指定的GPU设备上。
#### 接下来进行一些详细说明
#### 1、 在cuda设备上直接创建数据

In [1]:
import torch

In [2]:
#先检测设备上是否有GPU设备
has_cuda = torch.cuda.is_available()

#根据是否有GPU，创建设备
#torch.device()传入的设备名称字符串要求：Expected one of 
#cpu, cuda, ipu, xpu, mkldnn, opengl, opencl, ideep, hip, ve, ort, mps, xla, lazy, vulkan, meta, hpu
if has_cuda:
    my_device = torch.device('cuda')
    print('found cuda,return cuda device')
else:
    my_device = torch.device('cpu')
    print('No cuda,return cpu device')

No cuda,return cpu device


In [3]:
#在device上创建数据
train_data = torch.Tensor([1,2,3],device=my_device)

#判断创建的数据是否在cuda上
print(train_data.is_cuda)
#或者是
print(train_data.device)

False
cpu


In [4]:
#在device上创建torch.nn库下已经实现的网络模型
my_model1 = torch.nn.Conv2d(16,32,2,device=my_device)

#检查模型是否在cuda设备上：不能直接检查模型的位置，但是可以通过检查模型参数的位置来判断
next(my_model1.parameters()).device
next(my_model1.parameters()).is_cuda

False

In [5]:
#在device上创建自定义的网络模型，同样需要对网络层指定device=关键字参数
class My_Model(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.cnn = torch.nn.Sequential(torch.nn.Conv2d(16,32,2,device=my_device))
    def forward(self,x):
        return self.cnn(x)

#检查模型是否在cuda上
my_model2 = My_Model()
next(my_model2.parameters()).device

device(type='cpu')

In [6]:
#在device上创建损失函数：torch.nn下的已有的损失函数没有device参数，但是所有的损失函数都是继承自torch.nn.Module类
#而torch.nn.Module基类有一个cuda()函数，功能是将模型上的所有参数和缓存都移动到cuad上
#所以损失函数只能在创建后再转移到cuda上
my_loss = torch.nn.MSELoss()
my_loss.cuda()

MSELoss()

#### 2、先创建数据，模型，然后统一转移到GPU上

In [ ]:
#使用默认设备创建数据、网络模型、损失模型
data2 = torch.Tensor([1,2,3])
model2 = torch.nn.Conv2d(16,32,2)
loss2 = torch.nn.MSELoss()

cuda_list = []
cuda_list.append(data2)
cuda_list.append(model2)
cuda_list.append(loss2)

#使用.cuda()函数，将需要的内容转移到GPU上
for content in cuda_list:
    content.cuda()

In [ ]:
# 将网络模型在gpu上训练
model = Model()
model = model.cuda()

# 损失函数在gpu上训练
loss_fn = nn.CrossEntropyLoss()
loss_fn = loss_fn.cuda()

# 数据在gpu上训练
for data in dataloader:                        
	imgs, targets = data
	imgs = imgs.cuda()
	targets = targets.cuda()

#### 3、一些特殊的例子

In [ ]:
'''loss函数模型需要放到GPU上的例子'''

output = torch.randn(10, 10, requires_grad=True, device='cuda')
target = torch.randint(0, 10, (10,), device='cuda')

#这里为损失函数增加了权重，参与了矩阵运算
weight = torch.empty(10).uniform_(0, 1)
criterion = nn.CrossEntropyLoss(weight=weight)

loss = criterion(output, target) # error
> RuntimeError: Expected object of device type cuda but got device type cpu for argument 
#3 'weight' in call to _thnn_nll_loss_forward

criterion.cuda()
loss = criterion(output, target) # works

In [ ]:
'''一个使用cuda进行完整训练的例子，来自CSDN'''

# 以 CIFAR10 数据集为例，展示一下完整的模型训练套路，完成对数据集的分类问题

import torch
import torchvision

from torch import nn
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
import time

# 准备数据集
train_data = torchvision.datasets.CIFAR10(root="dataset", train=True, transform=torchvision.transforms.ToTensor(), download=True)
test_data = torchvision.datasets.CIFAR10(root="dataset", train=False, transform=torchvision.transforms.ToTensor(), download=True)

# 获得数据集的长度 len(), 即length
train_data_size = len(train_data)
test_data_size = len(test_data)

# 格式化字符串, format() 中的数据会替换 {}
print("训练数据集及的长度为: {}".format(train_data_size))
print("测试数据集及的长度为: {}".format(test_data_size))

# 利用DataLoader 来加载数据
train_dataloader = DataLoader(train_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

# 创建网络模型
class Model(nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.model = nn.Sequential(
            nn.Conv2d(3, 32, 5, 1, 2),
            nn.MaxPool2d(2),
            nn.Conv2d(32, 32, 5, 1, 2),
            nn.MaxPool2d(2),
            nn.Conv2d(32, 64, 5, 1, 2),
            nn.MaxPool2d(2),
            nn.Flatten(),
            nn.Linear(64*4*4, 64),
            nn.Linear(64, 10)
        )


    def forward(self, input):
        input = self.model(input)
        return input

model = Model()
if torch.cuda.is_available():
    model = model.cuda()                        # 在 GPU 上进行训练

# 创建损失函数
loss_fn = nn.CrossEntropyLoss()
if torch.cuda.is_available():
    loss_fn = loss_fn.cuda()                    # 在 GPU 上进行训练

# 优化器
learning_rate = 1e-2        # 1e-2 = 1 * (10)^(-2) = 1 / 100 = 0.01
optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate)

# 设置训练网络的一些参数
total_train_step = 0                        # 记录训练的次数
total_test_step = 0                         # 记录测试的次数
epoch = 10                                  # 训练的轮数

# 添加tensorboard
writer = SummaryWriter("logs_train")
start_time = time.time()                    # 开始训练的时间
for i in range(epoch):
    print("------第 {} 轮训练开始------".format(i+1))

    # 训练步骤开始
    for data in train_dataloader:
        imgs, targets = data
        if torch.cuda.is_available():
            imgs = imgs.cuda()
        targets = targets.cuda()            # 在gpu上训练
        outputs = model(imgs)               # 将训练的数据放入
        loss = loss_fn(outputs, targets)    # 得到损失值

        optimizer.zero_grad()               # 优化过程中首先要使用优化器进行梯度清零
        loss.backward()                     # 调用得到的损失，利用反向传播，得到每一个参数节点的梯度
        optimizer.step()                    # 对参数进行优化
        total_train_step += 1               # 上面就是进行了一次训练，训练次数 +1

        # 只有训练步骤是100 倍数的时候才打印数据，可以减少一些没有用的数据，方便我们找到其他数据
        if total_train_step % 100 == 0:
            end_time = time.time()          # 训练结束时间
            print("训练时间: {}".format(end_time - start_time))
            print("训练次数: {}, Loss: {}".format(total_train_step, loss))
            writer.add_scalar("train_loss", loss.item(), total_train_step)


    # 如何知道模型有没有训练好，即有咩有达到自己想要的需求
    # 我们可以在每次训练完一轮后，进行一次测试，在测试数据集上跑一遍，以测试数据集上的损失或正确率评估我们的模型有没有训练好

    # 顾名思义，下面的代码没有梯度，即我们不会利用进行调优
    total_test_loss = 0
    total_accuracy = 0                                      # 准确率
    with torch.no_grad():
        for data in test_dataloader:                        # 测试数据集中取数据
            imgs, targets = data
            if torch.cuda.is_available():
                imgs = imgs.cuda()                          # 在 GPU 上进行训练
                targets = targets.cuda()
            outputs = model(imgs)
            loss = loss_fn(outputs, targets)                # 这里的 loss 只是一部分数据(data) 在网络模型上的损失
            total_test_loss = total_test_loss + loss        # 整个测试集的loss
            accuracy = (outputs.argmax(1) == targets).sum() # 分类正确个数
            total_accuracy += accuracy                      # 相加

    print("整体测试集上的loss: {}".format(total_test_loss))
    print("整体测试集上的正确率: {}".format(total_accuracy / test_data_size))
    writer.add_scalar("test_loss", total_test_loss)
    writer.add_scalar("test_accuracy", total_accuracy / test_data_size, total_test_step)
    total_test_loss += 1                                    # 测试完了之后要 +1

    torch.save(model, "model_{}.pth".format(i))
    print("模型已保存")

writer.close()
